In [ ]:
import numpy as np
import statsmodels.api as sm
import math
import pandas as pd
from quantopian.research import run_pipeline, returns, get_pricing
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import CustomFactor, Returns, PercentChange, SimpleMovingAverage, BusinessDaysSincePreviousEvent, BusinessDaysUntilNextEvent
from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data import factset
from quantopian.pipeline.data.psychsignal import stocktwits
from quantopian.pipeline import factors, filters, classifiers
import alphalens
from scipy.stats.mstats import winsorize
from sklearn import preprocessing
from scipy.stats.mstats import gmean
from zipline.utils.numpy_utils import (
    repeat_first_axis,
    repeat_last_axis,
)
# from quantopian.pipeline.data.factset import RBICSFocus
from quantopian.pipeline.classifiers.fundamentals import Sector

import quantopian.pipeline.data.factset.estimates as fe


In [ ]:
#Factors taken from https://www.quantopian.com/posts/alpha-combination-via-clustering
WIN_LIMIT = 0
QL = 66
QLq = 80

def signalize(df):
    z = (df.rank() - 0.5)/df.count()
    return z.replace(np.nan, z.mean())

def preprocess(a):
    a = a.astype(np.float64)
    a[np.isinf(a)] = np.nan
    a = np.nan_to_num(a - np.nanmean(a))
    a = winsorize(a, limits=[WIN_LIMIT,WIN_LIMIT])
    
    return preprocessing.scale(a)

class Volatility(CustomFactor):  
    inputs = [USEquityPricing.close]  
    window_length = 22  
    def compute(self, today, assets, out, close):  
        # [0:-1] is needed to remove last close since diff is one element shorter  
        daily_returns = np.diff(close, axis = 0) / close[0:-1]  
        out[:] = daily_returns.std(axis = 0) * math.sqrt(252)
        
class MarketCap(CustomFactor):
    # Pre-declare inputs and window_length
    inputs = [USEquityPricing.close, Fundamentals.shares_outstanding]
    window_length = 1
    
    # Compute market cap value
    def compute(self, today, assets, out, close, shares):
        out[:] = close[-1] * shares[-1]


class FCFStability(CustomFactor):
    inputs = [Fundamentals.fcf_yield]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, fcf_yield):
        std = np.std([fcf_yield[-1], 
            fcf_yield[-QL], 
            fcf_yield[-QL*2],
            fcf_yield[-QL*3],
            fcf_yield[-QL*4],
            fcf_yield[-QL*5],
            fcf_yield[-QL*6],
            fcf_yield[-QL*7]], axis=0)

        out[:] = preprocess(-std)
        

class mean_rev(CustomFactor):   
    inputs = [USEquityPricing.high,USEquityPricing.low,USEquityPricing.close]
    window_length = 30
    window_safe = True
    def compute(self, today, assets, out, high, low, close):

        p = (high+low+close)/3

        m = len(close[0,:])
        n = len(close[:,0])
        b = np.zeros(m)
        a = np.zeros(m)

        for k in range(10,n+1):
            price_rel = np.nanmean(p[-k:,:],axis=0)/p[-1,:]
            wt = np.nansum(price_rel)
            b += wt*price_rel
            price_rel = 1.0/price_rel
            wt = np.nansum(price_rel)
            a += wt*price_rel

        out[:] = preprocess(b-a)

class growthscorestability(CustomFactor):
    inputs = [Fundamentals.growth_score]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):
        std = np.std([var[-1], 
            var[-QL], 
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
            var[-QL*7]], axis=0)

        out[:] = preprocess(-std)
                

class SalesGrowthStability(CustomFactor):
    inputs = [factset.Fundamentals.sales_gr_qf]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):
        mean = np.mean([var[-1], 
            var[-QL], 
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
            var[-QL*7]], axis=0)

        std = np.std([var[-1], 
            var[-QL], 
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
            var[-QL*7]], axis=0)
        
        out[:] = preprocess(mean/std)

        
        
class Gross_Income_Margin(CustomFactor):
    #Gross Income Margin:
    #Gross Profit divided by Net Sales
    #Notes:
    #High value suggests that the company is generating large profits
    inputs = [Fundamentals.cost_of_revenue, Fundamentals.total_revenue]
    window_length = 1
    window_safe = True
    def compute(self, today, assets, out, cost_of_revenue, sales):
        gross_income_margin = sales[-1]/sales[-1] - cost_of_revenue[-1]/sales[-1]
        out[:] = preprocess(-gross_income_margin)


class MaxGap(CustomFactor): 
    # the biggest absolute overnight gap in the previous 90 sessions
    inputs = [USEquityPricing.close] ; window_length = 90
    window_safe = True
    def compute(self, today, assets, out, close):
        abs_log_rets = np.abs(np.diff(np.log(close),axis=0))
        max_gap = np.max(abs_log_rets, axis=0)
        out[:] = preprocess(max_gap)

        
class fcf_growth_mean(CustomFactor):
    inputs=[
        Fundamentals.fcf_per_share,
        Fundamentals.shares_outstanding,
        Fundamentals.enterprise_value,]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, fcf, shares, ev):
        var = fcf*shares
        var[np.isinf(var)] = np.nan
        
        arr = [var[-1]/var[-QL] -1, 
            var[-QL]/var[-2*QL] -1,
            var[-QL*2]/var[-3*QL] -1, 
            var[-QL*3]/var[-4*QL] -1, 
            var[-QL*4]/var[-5*QL] -1, 
            var[-QL*5]/var[-6*QL] -1, 
            var[-QL*6]/var[-7*QL] -1]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean)
                
        
class TEM_GROWTH(CustomFactor):
    """
    TEM = standard deviation of past 6 quarters' reports
    """
    inputs=[factset.Fundamentals.capex_qf_asof_date,
        factset.Fundamentals.capex_qf,
        factset.Fundamentals.assets]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, asof_date, capex, total_assets):
        var = capex/total_assets
        var[np.isinf(var)] = np.nan
        arr = [var[-1]/var[-QL] -1, 
            var[-QL]/var[-2*QL] -1,
            var[-QL*2]/var[-3*QL] -1, 
            var[-QL*3]/var[-4*QL] -1, 
            var[-QL*4]/var[-5*QL] -1, 
            var[-QL*5]/var[-6*QL] -1, 
            var[-QL*6]/var[-7*QL] -1]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(-std)
        
        
class STA_Stability(CustomFactor):  
    inputs = [Fundamentals.operating_cash_flow,  
              Fundamentals.net_income_continuous_operations,  
              Fundamentals.total_assets]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, ocf, ni, ta):  
        ta = np.where(np.isnan(ta), 0, ta)  
        ocf = np.where(np.isnan(ocf), 0, ocf)  
        ni = np.where(np.isnan(ni), 0, ni)  
        var = abs(ni - ocf)/ ta
        
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean/std)
        
class FCFTA(CustomFactor):  
    inputs = [Fundamentals.free_cash_flow,  
             Fundamentals.total_assets]  
    window_length = 1
    window_safe = True
    def compute(self, today, assets, out, fcf, ta):  
        out[:] = preprocess(np.where(fcf[-1]/ta[-1]>0,1,0))

class Current_Ratio_Mean(CustomFactor):
    inputs=[Fundamentals.current_ratio]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean)

class Current_Ratio_Stability(CustomFactor):
    inputs=[Fundamentals.current_ratio]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean/std)
        
class pcf_ratio_stability(CustomFactor):
    inputs = [Fundamentals.pcf_ratio]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):
        std = np.std([1/var[-1], 
            1/var[-QL], 
            1/var[-QL*2],
            1/var[-QL*3],
            1/var[-QL*4],
            1/var[-QL*5],
            1/var[-QL*6],
            1/var[-QL*7]], axis=0)

        out[:] = preprocess(std)
        
class ps_ratio_stability(CustomFactor):
    inputs = [Fundamentals.ps_ratio]
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):
        std = np.std([1/var[-1], 
            1/var[-QL], 
            1/var[-QL*2],
            1/var[-QL*3],
            1/var[-QL*4],
            1/var[-QL*5],
            1/var[-QL*6],
            1/var[-QL*7]], axis=0)

        out[:] = preprocess(std)

class revenue_growth_mean(CustomFactor):  
    inputs = [Fundamentals.total_revenue]  
    window_length = QL*9
    window_safe = True
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
            var[-QL*7],            
               var[-QL*8]]

        arr = np.diff(arr, axis=0)/arr[1:]
        
        mean = np.mean(arr, axis=0)
        out[:] = preprocess(mean)

class revenue_growth_stability(CustomFactor):  
    inputs = [Fundamentals.revenue_growth]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean/std)

class pretax_margin_stability(CustomFactor):  
    inputs = [Fundamentals.pretax_margin]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean/std)


class NII_stability(CustomFactor):  
    inputs = [Fundamentals.net_interest_income]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean/std)
        
class NII_mean(CustomFactor):  
    inputs = [Fundamentals.net_interest_income]  
    window_length = QL*8
    window_safe = True
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6]]

        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean)
        

class NII_growth_stability(CustomFactor):  
    inputs = [Fundamentals.total_assets]    
    window_length = QL*9
    window_safe = True
    def compute(self, today, assets, out, var):
#         var  = fcf/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        arr = np.diff(arr, axis=0)/arr[1:]

        std = np.std(arr, axis=0)
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean/std)
        
class TA_growth_mean(CustomFactor):  
    inputs = [Fundamentals.total_assets]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, var):  
#         var  = fcf/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        arr = np.diff(arr, axis=0)/arr[1:]
        
        mean = np.mean(arr, axis=0)
        
        out[:] = preprocess(mean)

        
class TA_growth_Q0(CustomFactor):  
    inputs = [Fundamentals.total_assets]  
    window_length = QLq
    window_safe = True
    def compute(self, today, assets, out, var):  
        growth = ([var[-1]/var[-QLq]) - 1
        out[:] = preprocess(growth)
        
        
class LTA_mean(CustomFactor):  
    inputs = [Fundamentals.total_investments]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, ta):  
#         var  = loan/ta
        var = ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        arr = np.diff(arr, axis=0)/arr[1:]
        mean = np.mean(arr, axis=0)
        out[:] = preprocess(mean)

        
class TotalInv_stability(CustomFactor):   # Why negative contribution?
    inputs = [Fundamentals.total_investments]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, ta):  
#         var  = loan/ta
        var = ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        arr = np.diff(arr, axis=0)/arr[1:]
        mean = np.mean(arr, axis=0)
        std = np.std(arr, axis=0)
        out[:] = -preprocess(mean/std)
        
class TRTA_stability(CustomFactor):  
    inputs = [Fundamentals.trading_securities, Fundamentals.total_assets]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, ts, ta):  
        var  = ts/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
#         arr = np.diff(arr, axis=0)/arr[1:]
        mean = np.mean(arr, axis=0)
        std = np.std(arr, axis=0)
        out[:] = preprocess(mean/std)
        

class TRTA_mean(CustomFactor):  
    inputs = [Fundamentals.trading_securities, Fundamentals.total_assets]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, ts, ta):  
        var  = ts/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
#         arr = np.diff(arr, axis=0)/arr[1:]
        mean = np.mean(arr, axis=0)
#         std = np.std(arr, axis=0)
        out[:] = preprocess(mean)
        
        
class CRTa_stability(CustomFactor):  
    inputs = [Fundamentals.cost_of_revenue, Fundamentals.total_assets]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.mean(arr, axis=0)
        std = np.std(arr, axis=0)
        out[:] = -preprocess(mean/std)
        

class CR_growth_stability(CustomFactor):  
    inputs = [Fundamentals.cost_of_revenue]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, var):
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        arr = np.diff(arr, axis=0)/arr[1:]
        mean = np.mean(arr, axis=0)
        std = np.std(arr, axis=0)
        out[:] = -preprocess(mean/std)
        
class CrCrd_growth_mean(CustomFactor):  
    inputs = [Fundamentals.credit_card]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        arr = np.diff(arr, axis=0)/arr[1:]
        mean = np.mean(arr, axis=0)
        out[:] = preprocess(mean)

class CrCrd_growth_Q0(CustomFactor):  
    inputs = [Fundamentals.credit_card]  
    window_length = QLq
    window_safe = True
    
    def compute(self, today, assets, out, var):  
        growth = ([var[-1]/var[-QLq]) - 1 
        out[:] = preprocess(growth)
        
        
        
class DivIncmTa_mean(CustomFactor):  
    inputs = [Fundamentals.dividend_income, Fundamentals.total_assets]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.mean(arr, axis=0)
        out[:] = -preprocess(mean)
        
class EqtyIntrstEarng_growth_stability(CustomFactor):  
    inputs = [Fundamentals.earningsfrom_equity_interest_net_of_tax]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        arr = np.diff(arr, axis=0)/arr[1:]
        mean = np.mean(arr, axis=0)
        std = np.std(arr, axis=0)
        out[:] = preprocess(mean/std)
        
class EqtyIntrstEarng_growth_Q0(CustomFactor):  
    inputs = [Fundamentals.earningsfrom_equity_interest_net_of_tax]  
    window_length = QLq
    window_safe = True
    
    def compute(self, today, assets, out, var):  
        growth = ([var[-1]/var[-QLq]) - 1,
        out[:] = preprocess(growth)

class FnCTa_mean(CustomFactor):  
    inputs = [Fundamentals.fees_and_commissions, Fundamentals.total_assets]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.mean(arr, axis=0)
        out[:] = -preprocess(mean)

    
class FnCTa_stability(CustomFactor):  
    inputs = [Fundamentals.fees_and_commissions, Fundamentals.total_assets]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.mean(arr, axis=0)
        std = np.std(arr, axis=0)
        out[:] = preprocess(mean/std)

class FetgTa_mean(CustomFactor):  
    inputs = [Fundamentals.foreign_exchange_trading_gains, Fundamentals.total_assets]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
           
        
        mean = np.mean(arr, axis=0)
        out[:] = preprocess(mean)
        
class GainLoansTa_mean(CustomFactor):  
    inputs = [Fundamentals.gainon_saleof_loans, Fundamentals.total_assets]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.mean(arr, axis=0)
        out[:] = -preprocess(mean)

    
class GainLoansTa_stability(CustomFactor):  
    inputs = [Fundamentals.gainon_saleof_loans, Fundamentals.total_assets]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.mean(arr, axis=0)
        std = np.std(arr, axis=0)
        out[:] = preprocess(mean/std)
        
class IncomeDeposits_growth_stability(CustomFactor):  
    inputs = [Fundamentals.interest_income_from_deposits]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        arr = np.diff(arr, axis=0)/arr[1:]
        mean = np.mean(arr, axis=0)
        std = np.std(arr, axis=0)
        out[:] = -preprocess(mean/std)
        
class IncomeFdrlFundsTa_stability(CustomFactor):  
    inputs = [Fundamentals.interest_income_from_federal_funds_sold_and_securities_purchase_under_agreements_to_resell, Fundamentals.total_assets]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.mean(arr, axis=0)
        std = np.std(arr, axis=0)
        out[:] = -preprocess(mean/std)

        
class IncomeFdrlFunds_growth_stability(CustomFactor):  
    inputs = [Fundamentals.interest_income_from_federal_funds_sold_and_securities_purchase_under_agreements_to_resell]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        arr = np.diff(arr, axis=0)/arr[1:]
        mean = np.mean(arr, axis=0)
        std = np.std(arr, axis=0)
        out[:] = preprocess(mean/std)
        

class LeaseIncomeTa_mean(CustomFactor):  
    inputs = [Fundamentals.interest_income_from_leases, Fundamentals.total_assets]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.mean(arr, axis=0)
        out[:] = -preprocess(mean)
        

class IntrstIncmSec_growth_std(CustomFactor):  
    inputs = [Fundamentals.interest_income_from_securities]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        arr = np.diff(arr, axis=0)/arr[1:]
        std = np.std(arr, axis=0)
        out[:] = preprocess(-std)
        
class IntrstIncmNonOper_growth_mean(CustomFactor):  
    inputs = [Fundamentals.interest_income_non_operating]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        arr = np.diff(arr, axis=0)/arr[1:]
        mean = np.mean(arr, axis=0)
        out[:] = preprocess(mean)
        
class NonInrstIncm_growth_mean(CustomFactor):  
    inputs = [Fundamentals.non_interest_income]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        arr = np.diff(arr, axis=0)/arr[1:]
        mean = np.mean(arr, axis=0)
        out[:] = -preprocess(mean)

class NonInrstIncmTa_mean(CustomFactor):  
    inputs = [Fundamentals.non_interest_income, Fundamentals.total_assets]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.mean(arr, axis=0)
        out[:] = -preprocess(mean)
        
        
class PolicyHldrIntrst_growth_mean(CustomFactor):  
    inputs = [Fundamentals.policyholder_interest]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        arr = np.diff(arr, axis=0)/arr[1:]
        mean = np.mean(arr, axis=0)
        out[:] = preprocess(mean)
        
class PolicyDivTa_stability(CustomFactor):  
    inputs = [Fundamentals.policyholder_dividends, Fundamentals.total_assets]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.mean(arr, axis=0)
        std = np.std(arr, axis=0)
        out[:] = preprocess(mean/std)
        
        
class TotalPremiumTa_stability(CustomFactor):  
    inputs = [Fundamentals.total_premiums_earned, Fundamentals.total_assets]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.mean(arr, axis=0)
        std = np.std(arr, axis=0)
        out[:] = preprocess(mean/std)
        
class TradingGLTa_stability(CustomFactor):  
    inputs = [Fundamentals.trading_gain_loss, Fundamentals.total_assets]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.mean(arr, axis=0)
        std = np.std(arr, axis=0)
        out[:] = -preprocess(mean/std)
        
class CashTa_mean(CustomFactor):  
    inputs = [Fundamentals.cash, Fundamentals.total_assets]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.mean(arr, axis=0)
        out[:] = -preprocess(mean)
        
        
class CDTATa_stability(CustomFactor):  
    inputs = [Fundamentals.current_deferred_taxes_assets, Fundamentals.total_assets]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.mean(arr, axis=0)
        std = np.std(arr, axis=0)
        out[:] = -preprocess(mean/std)
        
class CDTATa_stability(CustomFactor):  
    inputs = [Fundamentals.current_deferred_taxes_assets, Fundamentals.total_assets]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.mean(arr, axis=0)
        std = np.std(arr, axis=0)
        out[:] = -preprocess(mean/std)
        
class CDTL_growth_mean(CustomFactor):  
    inputs = [Fundamentals.current_deferred_taxes_liabilities]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        arr = np.diff(arr, axis=0)/arr[1:]
        mean = np.mean(arr, axis=0)
        out[:] = preprocess(mean)

class CNPTa_mean(CustomFactor):  
    inputs = [Fundamentals.current_notes_payable, Fundamentals.total_assets]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.mean(arr, axis=0)
        std = np.std(arr, axis=0)
        out[:] = -preprocess(mean)
        
        
class CustAccnts_growth_dd(CustomFactor):  
    inputs = [Fundamentals.customer_accounts]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        arr = np.diff(arr, axis=0)/arr[1:]
        mean = np.mean(arr, axis=0)
        std = np.std(arr, axis=0)
        out[:] = preprocess(mean) + preprocess(-std)
        
class DepositsBank_growth_mean(CustomFactor):  
    inputs = [Fundamentals.depositsby_bank]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        arr = np.diff(arr, axis=0)/arr[1:]
        mean = np.mean(arr, axis=0)
        out[:] = preprocess(mean)

        
class FdrlsFundPurchTa_stability(CustomFactor):  
    inputs = [Fundamentals.federal_funds_purchased, Fundamentals.total_assets]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.mean(arr, axis=0)
        std = np.std(arr, axis=0)
        out[:] = preprocess(mean/std)

class FdrlFundsSold_growth_mean(CustomFactor):  
    inputs = [Fundamentals.federal_funds_sold]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, var):  
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        arr = np.diff(arr, axis=0)/arr[1:]
        mean = np.mean(arr, axis=0)
        out[:] = -preprocess(mean)

class ForeclosedAssetsTa_stability(CustomFactor):  
    inputs = [Fundamentals.foreclosed_assets, Fundamentals.total_assets]  
    window_length = QL*9
    window_safe = True
    
    def compute(self, today, assets, out, v, ta):  
        var  = v/ta
        arr = [var[-1],
            var[-QL],
            var[-QL*2],
            var[-QL*3],
            var[-QL*4],
            var[-QL*5],
            var[-QL*6],
           var[-QL*7],
         var[-QL*8]]
        
        mean = np.mean(arr, axis=0)
        std = np.std(arr, axis=0)
        out[:] = -preprocess(mean/std)
        
####Testing Var - banking related var growth and var/ta ratio
#ceded_premium - NO
#cost_of_revenue - WORKS (s/r)
#credit_card - WORkS_P(s)
#credit_losses_provision - NO
#credit_risk_provisions - No data
#dividend_income  WORKS(r)
#earningsfrom_equity_interest_net_of_tax - Yes (s)
#fees_and_commissions - WORKS (R)
#foreign_exchange_trading_gains - WORKS(R)
#gain_on_sale_of_security - NO
#gainon_investment_properties -  no data 
# Fundamentals.gainon_saleof_loans - WORKS (R)
#Fundamentals.interest_income - NO
#Fundamentals.interest_income_after_provision_for_loan_loss - NO
#Fundamentals.interest_income_from_deposits  - Works (s)
#Fundamentals.interest_income_from_federal_funds_sold_and_securities_purchase_under_agreements_to_resell (R)
#Fundamentals.interest_income_from_leases (R)
#Fundamentals.interest_income_from_securities (s)
#Fundamentals.interest_income_non_operating (s)
#Fundamentals.interest_income_other_operating_income - no data
#Fundamentals.investment_banking_profit - NO
#Fundamentals.non_interest_income - (s/R)
#Fundamentals.operating_income - WORKS but correlated
#Fundamentals.other_operating_income_total - no data
#Fundamentals.policyholder_dividends - WORKS
#Fundamentals.policyholder_interest - WORKS
#Fundamentals.securities_activities - NO
#Fundamentals.service_charge_on_depositor_accounts - NO
#Fundamentals.total_money_market_investments - NO
#Fundamentals.total_premiums_earned - WORKS (R)
#Fundamentals.total_revenue - WORKS  but correlated
#Fundamentals.trading_gain_loss -  WORKS but little data (Rs)
#Fundamentals.underwriting_expenses - NO
#advancesfrom_central_banks - No data
#assets_held_for_sale - NO
#bank_loans_total - No data
#cash  - WORKS (R)

# x = Fundamentals.claims_outstanding

# class Var_growth_mean(CustomFactor):  
#     inputs = [x]  
#     window_length = QL*9
#     window_safe = True
    
#     def compute(self, today, assets, out, var):  
#         arr = [var[-1],
#             var[-QL],
#             var[-QL*2],
#             var[-QL*3],
#             var[-QL*4],
#             var[-QL*5],
#             var[-QL*6],
#            var[-QL*7],
#          var[-QL*8]]
        
#         arr = np.diff(arr, axis=0)/arr[1:]
#         mean = np.mean(arr, axis=0)
#         out[:] = preprocess(mean)

# class Var_growth_std(CustomFactor):  
#     inputs = [x]  
#     window_length = QL*9
#     window_safe = True
    
#     def compute(self, today, assets, out, var):  
#         arr = [var[-1],
#             var[-QL],
#             var[-QL*2],
#             var[-QL*3],
#             var[-QL*4],
#             var[-QL*5],
#             var[-QL*6],
#            var[-QL*7],
#          var[-QL*8]]
        
#         arr = np.diff(arr, axis=0)/arr[1:]
#         std = np.std(arr, axis=0)
#         out[:] = preprocess(-std)

        
# class Var_growth_stability(CustomFactor):  
#     inputs = [x]  
#     window_length = QL*9
#     window_safe = True
    
#     def compute(self, today, assets, out, var):  
#         arr = [var[-1],
#             var[-QL],
#             var[-QL*2],
#             var[-QL*3],
#             var[-QL*4],
#             var[-QL*5],
#             var[-QL*6],
#            var[-QL*7],
#          var[-QL*8]]
        
#         arr = np.diff(arr, axis=0)/arr[1:]
#         mean = np.mean(arr, axis=0)
#         std = np.std(arr, axis=0)
#         out[:] = preprocess(mean/std)


# class VarTa_mean(CustomFactor):  
#     inputs = [x, Fundamentals.total_assets]  
#     window_length = QL*9
#     window_safe = True
    
#     def compute(self, today, assets, out, v, ta):  
#         var  = v/ta
#         arr = [var[-1],
#             var[-QL],
#             var[-QL*2],
#             var[-QL*3],
#             var[-QL*4],
#             var[-QL*5],
#             var[-QL*6],
#            var[-QL*7],
#          var[-QL*8]]
        
#         mean = np.mean(arr, axis=0)
#         out[:] = preprocess(mean)

# class VarTa_std(CustomFactor):  
#     inputs = [x, Fundamentals.total_assets]  
#     window_length = QL*9
#     window_safe = True
    
#     def compute(self, today, assets, out, v, ta):  
#         var  = v/ta
#         arr = [var[-1],
#             var[-QL],
#             var[-QL*2],
#             var[-QL*3],
#             var[-QL*4],
#             var[-QL*5],
#             var[-QL*6],
#            var[-QL*7],
#          var[-QL*8]]
        
#         std = np.std(arr, axis=0)
#         out[:] = preprocess(-std)
    
# class VarTa_stability(CustomFactor):  
#     inputs = [x, Fundamentals.total_assets]  
#     window_length = QL*9
#     window_safe = True
    
#     def compute(self, today, assets, out, v, ta):  
#         var  = v/ta
#         arr = [var[-1],
#             var[-QL],
#             var[-QL*2],
#             var[-QL*3],
#             var[-QL*4],
#             var[-QL*5],
#             var[-QL*6],
#            var[-QL*7],
#          var[-QL*8]]
        
#         mean = np.mean(arr, axis=0)
#         std = np.std(arr, axis=0)
#         out[:] = preprocess(mean/std)
        
        
# sector = RBICSFocus.l1_name.latest
# base_universe = QTradableStocksUS() & sector.eq('Finance')

base_universe = QTradableStocksUS() & Sector().element_of( [103, 104])

# guidance_days_prev = BusinessDaysSincePreviousEvent(inputs=[fe.Actuals.slice('SALES', 'qf', 0).asof_date])  
# guidance_days_next = BusinessDaysUntilNextEvent(inputs=[fe.Actuals.slice('SALES', 'qf', 0).asof_date])  

mkt_cap = MarketCap(mask = base_universe)
vol = Volatility(mask = base_universe)

alpha4 = mean_rev(mask = base_universe)        
alpha12 = Gross_Income_Margin(mask = base_universe)        
alpha13 = MaxGap(mask = base_universe)        
alpha25 = FCFTA(mask = base_universe)        
alpha43 = FCFStability(mask = base_universe)
alpha44 = growthscorestability(mask = base_universe)
alpha46 = SalesGrowthStability(mask = base_universe)
alpha52 = fcf_growth_mean(mask = base_universe)
alpha54 = TEM_GROWTH(mask = base_universe)
alpha59 = STA_Stability(mask = base_universe)
alpha65 = Current_Ratio_Mean(mask = base_universe)
alpha71 = pcf_ratio_stability(mask = base_universe)
alpha72 = ps_ratio_stability(mask = base_universe)
alpha76 = revenue_growth_mean(mask = base_universe)
alpha78 = revenue_growth_stability(mask = base_universe)
alpha84 = pretax_margin_stability(mask = base_universe)


#Pure Financials Factors

alpha85 = NII_growth_stability(mask = base_universe)
alpha86 = TA_growth_mean(mask = base_universe)

# alpha87 = LTA_mean(mask = base_universe)
alpha88 = TotalInv_stability(mask = base_universe) # Why this is negative contributor

# alpha89 = TRTA_mean(mask = base_universe)
alpha90 = TRTA_stability(mask = base_universe)

alphaI1 = CR_growth_stability(mask = base_universe)
alphaI2 = CRTa_stability(mask = base_universe)

alphaI3 = CrCrd_growth_mean(mask = base_universe)
alphaI3a = CrCrd_growth_Q0(mask = base_universe)



alphaI4 = DivIncmTa_mean(mask = base_universe)
alphaI5 = EqtyIntrstEarng_growth_stability(mask = base_universe)

alphaI6 = FnCTa_mean(mask = base_universe)
alphaI7 = FnCTa_stability(mask = base_universe)

alphaI8 = FetgTa_mean(mask = base_universe)

alphaI9 = GainLoansTa_mean(mask = base_universe)
alphaI10 = GainLoansTa_stability(mask = base_universe)
alphaI11 = IncomeDeposits_growth_stability(mask = base_universe)

alphaI12 = IncomeFdrlFundsTa_stability(mask = base_universe)
alphaI13 = IncomeFdrlFunds_growth_stability(mask = base_universe)

alphaI14 = LeaseIncomeTa_mean(mask = base_universe)

alphaI15 = IntrstIncmSec_growth_std(mask = base_universe)

alphaI16 = IntrstIncmNonOper_growth_mean(mask = base_universe)
alphaI17 = NonInrstIncmTa_mean(mask = base_universe)
alphaI18 = NonInrstIncm_growth_mean(mask = base_universe)

alphaI19 = PolicyHldrIntrst_growth_mean(mask = base_universe)
alphaI20 = PolicyDivTa_stability(mask = base_universe)

alphaI21 = TotalPremiumTa_stability(mask = base_universe) 
alphaI22 = TradingGLTa_stability(mask = base_universe) 

alphaI23 = CashTa_mean(mask = base_universe) 

alphaI24 = CDTATa_stability(mask = base_universe) #No

alphaI25 = CDTL_growth_mean(mask = base_universe)

alphaI26 = CNPTa_mean(mask = base_universe)

alphaI27 = CustAccnts_growth_dd(mask = base_universe)

alphaI28 = DepositsBank_growth_mean(mask = base_universe)

alphaI29 = FdrlsFundPurchTa_stability(mask = base_universe)

alphaI30 = FdrlFundsSold_growth_mean(mask = base_universe)

alphaI31 = ForeclosedAssetsTa_stability(mask = base_universe)
# alpha91 = Var_growth_mean(mask = base_universe)
# alpha92 = Var_growth_std(mask = base_universe)
# alpha93 = Var_growth_stability(mask = base_universe)

# alpha94 = VarTa_mean(mask = base_universe)
# alpha95 = VarTa_std(mask = base_universe)
# alpha96 = VarTa_stability(mask = base_universe)


pipe_alpha_factors = Pipeline(
    columns={
'vol': vol,
'mkt_cap':mkt_cap,
        
#         'alpha4':alpha4,    #WL

#         'alpha12':alpha12,
#         'alpha13':alpha13, #WL
        
#         'alpha25':alpha25,

#         'alpha43': alpha43,
#         'alpha44':alpha44,
        
#         'alpha46': alpha46,
        
#         'alpha52': alpha52,

#         'alpha54': alpha54,

#         'alpha59': alpha59,

#         'alpha65':alpha65,

#         'alpha71':alpha71,
#         'alpha72':alpha72, #WL
#         'alpha76': alpha76, #WL

#         'alpha78': alpha78,

#         'alpha84': alpha84,
        
#         'alpha85': alpha85,
#         'alpha86': alpha86, #WL
        
#         'alpha88': alpha88,

#         'alpha90': alpha90,
    
#         'alphaI1': alphaI1,
#         'alphaI2': alphaI2,
#         'alphaI3': alphaI3,
#         'alphaI4': alphaI4,
#         'alphaI5': alphaI5,
        
#         'alphaI6': alphaI6,
#         'alphaI7': alphaI7,
#         'alphaI8': alphaI8,
        
#         'alphaI9': alphaI9,  #WL
#         'alphaI10': alphaI10,
#         'alphaI11': alphaI11,   #WL
            
#         'alphaI12': alphaI12,
#         'alphaI13': alphaI13,
        
#         'alphaI14': alphaI14,
        
#         'alphaI15': alphaI15,
        
#         'alphaI16': alphaI16,
        
#         'alphaI17': alphaI17 ,
#         'alphaI18': alphaI18,   #WL
        
#         'alphaI19': alphaI19,
#         'alphaI20': alphaI20,
#         'alphaI21': alphaI21,
#         'alphaI22': alphaI22,
        
#         'alphaI23': alphaI23,   #WL
#         'alphaI24': alphaI24,
        
#         'alphaI25': alphaI25,
#                 'alphaI26': alphaI26,
#                 'alphaI27': alphaI27,    #WL
#                 'alphaI28': alphaI28,

#                 'alphaI29': alphaI29,     #WL
#                 'alphaI30': alphaI30,
#                 'alphaI31': alphaI31,

#         'alpha91': alpha91,
#         'alpha92': alpha92,
#         'alpha93': alpha93,        

#         'alpha94': alpha94,
#         'alpha95': alpha95,
#         'alpha96': alpha96,

#         'guidance_days_next': guidance_days_next,
#         'guidance_days_prev': guidance_days_prev
    },
    screen=base_universe
)   

sd = '2008-06-01'
ed = '2018-09-01'
output = run_pipeline(pipe_alpha_factors, sd, ed)

In [ ]:
assets = output.index.levels[1].unique()
# We need to get a little more pricing data than the 
# length of our factor so we can compare forward returns.
# We'll tack on another month in this example.
pricing = get_pricing(assets, start_date=sd, end_date=ed, fields='close_price')


In [ ]:
# print data['alpha87'].tail(10)

In [ ]:
# factor_names = ['alpha4', 'alpha43', 'alpha46', 'alpha52', 
#                 'alpha12', 'alpha13', 'alpha25', 'alpha44', 
#                   'alpha46', 'alpha54', 'alpha59', 'alpha65',
#                 'alpha71', 'alpha72', 'alpha76', 'alpha78', 'alpha84',
#                 'alpha85', 'alpha86', 'alpha88', 'alpha90',
                
# #                 'alpha91', 'alpha92', 'alpha93', 'alpha94', 'alpha95', 'alpha96',
                
#                 'alphaI1', 'alphaI2', 'alphaI3', 'alphaI4', 
#                 'alphaI5', 'alphaI6', 'alphaI7', 'alphaI8', 
#                 'alphaI9', 'alphaI10', 'alphaI11', 'alphaI12',
#                 'alphaI13', 'alphaI14', 'alphaI15', 'alphaI16', 
#                 'alphaI17', 'alphaI18', 'alphaI19', 'alphaI20', 
#                 'alphaI21', 'alphaI22', 'alphaI23', 'alphaI24', 'alphaI25',
#                 'alphaI26', 'alphaI27', 'alphaI28', 'alphaI29', 'alphaI30', 'alphaI31'
#                ]


# YES - 'alpha4', 'alpha13', 'alpha72', 'alpha76', 'alpha86', 'alphaI9', 
# 'alphaI11', 'alphaI18', 'alphaI23', 'alphaI27', 'alphaI29'

# NO - 'alpha12', 'alpha25', 'alpha43', 'alpha44', 'alpha46', 'alpha52', 'alpha54', 'alpha59', 
# 'alpha65', 'alpha71', 'alpha78', 'alpha84', 'alpha85', 'alpha88', 'alpha90', 'alphaI1', 
# 'alphaI2', 'alphaI3', 'alphaI4', 'alphaI5', 'alphaI6', 'alphaI7', 'alphaI8', 'alphaI10',
# 'alphaI12', 'alphaI13', 'alphaI14', 'alphaI15', 'alphaI16', 'alphaI17', 'alphaI19', 'alphaI20', 
# 'alphaI21', 'alphaI22', 'alphaI24', 'alphaI25', 'alphaI26', 'alphaI28', 'alphaI30', 'alphaI31'

factor_names = ['alpha4', 'alpha13', 'alpha72', 'alpha76', 'alpha86', 'alphaI9', 'alphaI11', 'alphaI18', 'alphaI23', 'alphaI27', 'alphaI29']

def wmean(name):
    return (output['mkt_cap']*output[name]).sum(level=0)/output['mkt_cap'].mean(level=0)

data = pd.DataFrame()
# data = output[['guidance_days_prev','guidance_days_next']]

#Demean the alphas
for name in factor_names:
    output['s'+name] = output[name]
    data[name] = output['s'+name].sub(output['s'+name].mean(level=0), level=0)
#     output[name] = output[name].sub(wmean(name), level=0)



In [ ]:
returns = pricing.pct_change(periods=5).shift(-5)
#Demean the returns
returns = returns.sub(returns.mean(axis=1), axis=0).stack()
data = pd.concat([returns.rename('returns'), data], axis=1).dropna()

In [ ]:
data.reset_index(inplace=True)
data = data.rename(columns={'level_0':'dates', 'level_1': 'sid'})

In [ ]:
significant_factor_names = ['alpha4', 'alpha13', 'alpha76', 'alpha86', 'alphaI9', 'alphaI11', 'alphaI18', 'alphaI23', 'alphaI27', 'alphaI29']

# ['alpaha72' ,'alpha76', 'alpha4', 'alpha13']
est = sm.OLS(
    100*data[['returns']], 
        data[significant_factor_names])

est2 = est.fit(cov_type='cluster', cov_kwds={'groups': data['dates']})

print est2.summary()

In [ ]:
d = data.set_index(['dates', 'sid'])
net_alpha = d['alpha4'] + d['alpha13'] + d['alpha76'] + d['alpha86'] + d['alphaI9'] + d['alphaI11'] + d['alphaI18'] + d['alphaI23'] + d['alphaI27'] + d['alphaI29']
factor_data = alphalens.utils.get_clean_factor_and_forward_returns(net_alpha,
                                                                   pricing,
                                                                   quantiles=5,
                                                                   periods=(5,10))

# alphalens.tears.create_returns_tear_sheet(factor_data)
# alphalens.tears.create_full_tear_sheet(factor_data)
alphalens.tears.create_summary_tear_sheet(factor_data)